In [1]:
import timm
from torchsummary import summary

In [2]:
base_model = timm.create_model(
            "tf_efficientnet_b4_ns", pretrained=True, num_classes=1
        )
summary(base_model, (3,299,299), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2dSame: 1-1                             [-1, 48, 150, 150]        1,296
├─BatchNorm2d: 1-2                            [-1, 48, 150, 150]        96
├─SwishMe: 1-3                                [-1, 48, 150, 150]        --
├─Sequential: 1-4                             [-1, 448, 10, 10]         --
|    └─Sequential: 2-1                        [-1, 24, 150, 150]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 24, 150, 150]        2,940
|    |    └─DepthwiseSeparableConv: 3-2       [-1, 24, 150, 150]        1,206
|    └─Sequential: 2-2                        [-1, 32, 75, 75]          --
|    |    └─InvertedResidual: 3-3             [-1, 32, 75, 75]          11,878
|    |    └─InvertedResidual: 3-4             [-1, 32, 75, 75]          18,120
|    |    └─InvertedResidual: 3-5             [-1, 32, 75, 75]          18,120
|    |    └─InvertedResidual: 3-6             [-1, 32, 75, 75]          18

Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2dSame: 1-1                             [-1, 48, 150, 150]        1,296
├─BatchNorm2d: 1-2                            [-1, 48, 150, 150]        96
├─SwishMe: 1-3                                [-1, 48, 150, 150]        --
├─Sequential: 1-4                             [-1, 448, 10, 10]         --
|    └─Sequential: 2-1                        [-1, 24, 150, 150]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 24, 150, 150]        2,940
|    |    └─DepthwiseSeparableConv: 3-2       [-1, 24, 150, 150]        1,206
|    └─Sequential: 2-2                        [-1, 32, 75, 75]          --
|    |    └─InvertedResidual: 3-3             [-1, 32, 75, 75]          11,878
|    |    └─InvertedResidual: 3-4             [-1, 32, 75, 75]          18,120
|    |    └─InvertedResidual: 3-5             [-1, 32, 75, 75]          18,120
|    |    └─InvertedResidual: 3-6             [-1, 32, 75, 75]          18